In [1]:
import time
from collections import deque, namedtuple
import gym
import numpy as np
import PIL.Image
import tensorflow as tf
import utils

from pyvirtualdisplay import Display
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Input
from tensorflow.keras.losses import MSE
from tensorflow.keras.optimizers import Adam
from force3env1 import Force3Env

In [2]:
# Définissez la graine aléatoire pour TensorFlow
tf.random.set_seed(utils.SEED)

In [3]:
MEMORY_SIZE = 100_000     # size of memory buffer
GAMMA = 0.995             # discount factor
ALPHA = 1e-3              # learning rate  
NUM_STEPS_FOR_UPDATE = 4  # perform a learning update every C time steps

In [4]:
env = Force3Env()

In [5]:
state_size = env.observation_space.shape
num_actions = env.action_space[0].n * env.action_space[1].n * env.action_space[2].n


print('State Shape:', state_size)
print('Number of actions:', num_actions)

State Shape: (3, 3, 1)
Number of actions: 243


In [6]:
current_state = env.reset()

print(current_state)

[[2 2 2]
 [2 0 2]
 [2 2 2]]


In [7]:
# Select an action
action = (0, 1, 3)

# Run a single time step of the environment's dynamics with the given action.
next_state, reward3, done, _ = env.step(action, False)

# Replace the `current_state` with the state after the action is taken
current_state = next_state

In [8]:
print(current_state)

[[ 2  2  2]
 [-1  0  2]
 [ 2  2  2]]


In [77]:
def create_dqn_model(input_shape, num_actions):
    model = Sequential()

    # Première couche convolutive
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    
    # Deuxième couche convolutive
    model.add(Conv2D(64, (3, 3), activation='relu'))

    # Aplatir les données pour les couches denses
    model.add(Flatten())

    # Première couche dense
    model.add(Dense(64, activation='relu'))

    # Deuxième couche dense
    model.add(Dense(128, activation='relu'))

    # Couche de sortie
    model.add(Dense(num_actions, activation='linear'))

    # Compiler le modèle
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model